<a href="https://colab.research.google.com/github/pannettonne/Portfolio/blob/main/LaSalle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import importlib.util
import subprocess
#import pkg_resources
import sys
print("Empiezo")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


import os
import yagmail
from premailer import Premailer

#import pyjuice as juice
import css_inline

import re
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def minify_html(html: str) -> str:
    html = re.sub(r'>\s+<', '><', html)  # elimina espacios/saltos entre etiquetas
    html = re.sub(r'\s{2,}', ' ', html)  # colapsa espacios repetidos
    return html.strip()

def norm_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s).replace("\xa0", " ")
    s = s.replace("–", "-").replace("—", "-").replace("\u2011","-").replace("\u200b","")
    return " ".join(s.split())

#os.environ['KEYRING_BACKEND'] = 'keyrings.alt.chainer.ChainerBackend'

# *** Manteniendo como lo tenías (si usas app password) ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, JavascriptException
import time

from webdriver_manager.chrome import ChromeDriverManager

# --- WebDriver ---
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'
# logs de navegador (por si quieres ver errores JS)
chrome_options.set_capability('goog:loggingPrefs', {'browser': 'ALL'})

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=chrome_options)

# ====== METEO (Open-Meteo) ======
import json
from datetime import datetime as dtmod
def _http_get(url):
    try:
        import requests
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        return r.text
    except Exception:
        # fallback sin requests
        import urllib.request
        with urllib.request.urlopen(url, timeout=15) as resp:
            return resp.read().decode("utf-8")

def _weekday_es(dt: datetime) -> str:
    # 0..6 = Lunes..Domingo
    nombres = ["Lunes","Martes","Miércoles","Jueves","Viernes","Sábado","Domingo"]
    return nombres[dt.weekday()]

def _titulo_dia_con_nombre(fecha_str: str) -> str:
    """
    Recibe 'dd/mm[/yyyy]' y devuelve 'Miércoles 01/10/2025'.
    Si falta el año, usa el año actual. Si el año viene en 2 dígitos, lo convierte a 20xx.
    """
    m = re.match(r"^\s*(\d{1,2})/(\d{1,2})(?:/(\d{2,4}))?\s*$", fecha_str)
    if not m:
        return fecha_str  # fallback tal cual
    d, mth, y = int(m.group(1)), int(m.group(2)), m.group(3)
    if y is None:
        y = datetime.now().year
    else:
        y = int(y)
        if y < 100:
            y += 2000
    dt = datetime(year=y, month=mth, day=d)
    return f"{_weekday_es(dt)} {dt.strftime('%d/%m/%Y')}"

def _wcode_text_emoji(code: int) -> str:
    m = {
        0: ("Despejado", "☀️"),
        1: ("Más bien despejado", "🌤️"),
        2: ("Parcialmente nublado", "⛅"),
        3: ("Nublado", "☁️"),
        45: ("Niebla", "🌫️"), 48: ("Niebla con escarcha", "🌫️"),
        51: ("Llovizna débil", "🌦️"), 53: ("Llovizna", "🌦️"), 55: ("Llovizna intensa", "🌦️"),
        56: ("Llovizna helada", "🌧️"), 57: ("Llovizna helada", "🌧️"),
        61: ("Lluvia débil", "🌦️"), 63: ("Lluvia", "🌧️"), 65: ("Lluvia intensa", "🌧️"),
        66: ("Lluvia helada", "🌨️"), 67: ("Lluvia helada", "🌨️"),
        71: ("Nieve débil", "🌨️"), 73: ("Nieve", "🌨️"), 75: ("Nieve intensa", "🌨️"),
        77: ("Granos de nieve", "🌨️"),
        80: ("Chubascos débiles", "🌧️"), 81: ("Chubascos", "🌧️"), 82: ("Chubascos intensos", "🌧️"),
        85: ("Chubascos de nieve", "🌨️"), 86: ("Chubascos de nieve intensos", "🌨️"),
        95: ("Tormenta", "⛈️"), 96: ("Tormenta con granizo", "⛈️"), 99: ("Tormenta con granizo fuerte", "⛈️"),
    }
    return " ".join(m.get(code, ("", "❓")))

def meteo_moncloa_semana_html() -> str:
    print("METEO")
    lat, lon = 40.437, -3.732
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        "&daily=weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max"
        "&timezone=Europe%2FMadrid"
    )
    try:
        txt = _http_get(url)
        data = json.loads(txt)
        d = data["daily"]
        fechas = d["time"]
        wcode = d["weathercode"]
        tmax = d["temperature_2m_max"]
        tmin = d["temperature_2m_min"]
        prcp = d["precipitation_sum"]
        wmax = d["windspeed_10m_max"]

        filas = []
        for i in range(len(fechas)):
            dt = dtmod.strptime(fechas[i], "%Y-%m-%d")
            dia_nombre = ["Lun","Mar","Mié","Jue","Vie","Sáb","Dom"][dt.weekday()]
            fecha_fmt = dt.strftime("%d/%m")
            wtxt = _wcode_text_emoji(int(wcode[i]))
            filas.append(
                f"<tr>"
                f"<td>{dia_nombre} {fecha_fmt}</td>"
                f"<td>{wtxt}</td>"
                f"<td style='text-align:right;'>{round(tmax[i])}°</td>"
                f"<td style='text-align:right;'> {round(tmin[i])}°</td>"
                f"<td style='text-align:right;'>{prcp[i]:.1f} mm</td>"
                f"<td style='text-align:right;'>{round(wmax[i])} km/h</td>"
                f"</tr>"
            )

        tabla = (
            "<table style='width:100%; border-collapse:collapse;' class='table table-hover table-bordered'>"
            "<thead>"
            "<tr>"
            "<th>Día</th><th>Estado</th><th>Máx</th><th>Mín</th><th>Precip</th><th>Viento máx</th>"
            "</tr>"
            "</thead>"
            "<tbody>"
            + "".join(filas) +
            "</tbody></table>"
        )
        bloque = (
            "<h1 style='font-weight:bold; margin-top:16px;'>Tiempo en Moncloa (Madrid) · Semana</h1>"
            + tabla
        )
        return bloque
    except Exception as e:
        return f"<p><em>No se pudo cargar el tiempo (Open-Meteo): {e}</em></p>"
# ====== FIN METEO ======

def dump_debug(name="debug"):
    try:
        path_png = f"/tmp/{name}.png"
        path_html = f"/tmp/{name}.html"
        driver.save_screenshot(path_png)
        with open(path_html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[DEBUG] Screenshot: {path_png}")
        print(f"[DEBUG] HTML: {path_html}")
        sys.stdout.flush()
    except Exception as e:
        print(f"[DEBUG] No se pudo guardar debug: {e}", flush=True)

# -------- Helpers UI --------
def esperar_tabla(timeout=20):
    print("[DBG] Esperando tabla en #div_padres…")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
        ))
    )
    print("[DBG] Tabla detectada.")

def click_vista_diaria(timeout=10):
    print("[DBG] Click 'Vista diaria'…")
    btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(normalize-space(.), "Vista diaria")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
    esperar_tabla()
    print("[DBG] Click 'Vista diaria' OK.")

def click_vista_semanal(timeout=10):
    print("[DBG] Click 'Vista semanal'…")
    btn = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(normalize-space(.), "Vista semanal")]'))
    )
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
    driver.execute_script("arguments[0].click();", btn)
    esperar_tabla()
    print("[DBG] Click 'Vista semanal' OK.")

def set_fecha_en_input_y_disparar_change(fecha_str: str):
    print(f"[DBG] Set #fecha_incidencia = {fecha_str} + change()…")
    js = """
        const f = arguments[0];
        const inp = document.getElementById('fecha_incidencia');
        if (!inp) return 'NO_INPUT';
        inp.value = f;
        if (window.jQuery) {
            jQuery(inp).trigger('change');
            return 'JQ_CHANGE';
        } else {
            const ev = new Event('change', {bubbles:true});
            inp.dispatchEvent(ev);
            return 'NATIVE_CHANGE';
        }
    """
    res = driver.execute_script(js, fecha_str)
    print(f"[DBG] Resultado set/trigger: {res}")

def esperar_header_de_fecha(fecha_str: str, timeout=25):
    print(f"[DBG] Esperando <th> con '{fecha_str}'…")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((
            By.XPATH, f'//*[@id="div_padres"]//th[contains(normalize-space(.), "{fecha_str}")]'
        ))
    )
    print("[DBG] Header con la fecha detectado.")

def get_tabla_html() -> str:
    tabla_elem = driver.find_element(
        By.XPATH,
        '//*[@id="div_padres"]//table[contains(@class,"table") and contains(@class,"table-bordered")]'
    )
    html = tabla_elem.get_attribute("outerHTML")
    print(f"[DBG] Longitud HTML tabla: {len(html)}")
    return html

def quitar_notas_personales(html: str) -> str:
    try:
        soup = BeautifulSoup(html, "html.parser")
        for tr in soup.select("tr"):
            first_th = tr.find("th")
            if not first_th:
                continue
            label = norm_text(first_th.get_text(" ", strip=True)).lower()
            if label == "notas personales":
                tr.decompose()
        return str(soup)
    except Exception:
        return html

# ========= NUEVO: PARSER ESTADO-MÁQUINA (Tareas/Planificación como filas) =========

# Detecta fechas tipo 01/10/2025 o 01/10/25 o 01/10
_RE_FECHA = re.compile(r"\b([0-3]?\d/[01]?\d(?:/\d{2,4})?)\b", re.I)

def _es_fila_header_de_dia(tr) -> str | None:
    ths = tr.find_all("th", recursive=False)
    tds = tr.find_all("td", recursive=False)
    if len(ths) == 1 and len(tds) == 0:
        txt = norm_text(ths[0].get_text(" ", strip=True))
        m = _RE_FECHA.search(txt)
        if m:
            return m.group(1)
    return None

def _celda_html(td) -> str:
    try:
        return td.decode_contents().strip()
    except Exception:
        return norm_text(td.get_text(" ", strip=True))

def _limpia_rotulo(rot: str) -> str:
    rot = norm_text(rot)
    # quita "Asignatura:" si aparece
    rot = re.sub(r"(?i)^asignatura:\s*", "", rot)
    # quita el prefijo "NCA 2º EP)" (variantes: 2o, espacios, etc.)
    rot = re.sub(r"^NCA\s*2[ºo]\s*EP\)\s*", "", rot, flags=re.IGNORECASE)
    rot = re.sub(r"^2[ºo]\s*B\)\s*", "", rot, flags=re.IGNORECASE)
    return rot

def parsear_bloques_por_dia(html_tabla: str):
    """
    Devuelve: [{'fecha': 'dd/mm[/yyyy]', 'filas': [{'asignatura','tareas','plan'}, ...]}, ...]
    Soporta dos formatos:
      A) MATRIZ (semanal):
         - tr.alert.alert-info -> cabecera horizontal con varias asignaturas (varios <th>), puede haber un <td> vacío inicial
         - tr 'Tareas'   -> varios <td> (uno por asignatura)
         - tr 'Planificación' -> varios <td> (uno por asignatura)
      B) VERTICAL (diaria):
         - <tr><th>ASIGNATURA</th>[<td>tareas</td>[<td>plan</td>]]</tr>
         - Filas siguientes "Tareas" / "Planificación" completan la asignatura actual
    """
    soup = BeautifulSoup(html_tabla, "html.parser")
    tabla = soup.find("table")
    if not tabla:
        return []

    bloques = []
    actual_bloque = None

    # Estado (para formato VERTICAL)
    asignatura_actual = None

    # Estado (para formato MATRIZ)
    cabecera_cols = None      # lista de nombres de asignatura (por columnas)
    tareas_cols   = None      # lista de html por columna (mismo len que cabecera_cols)
    plan_cols     = None

    def _push_asignatura_vertical():
        nonlocal asignatura_actual
        if asignatura_actual and (asignatura_actual.get('asignatura') or asignatura_actual.get('tareas') or asignatura_actual.get('plan')):
            asignatura_actual['tareas'] = asignatura_actual.get('tareas', '') or ''
            asignatura_actual['plan']   = asignatura_actual.get('plan', '') or ''
            actual_bloque['filas'].append(asignatura_actual)
        asignatura_actual = None

    def _cerrar_bloque_matriz_si_hay():
        """Si hemos recogido cabecera + tareas/plan (formato matriz), volcamos filas al bloque actual."""
        nonlocal cabecera_cols, tareas_cols, plan_cols
        if actual_bloque and cabecera_cols:
            # Rellena con cadenas vacías si faltan
            n = len(cabecera_cols)
            t = (tareas_cols or [])
            p = (plan_cols or [])
            if len(t) < n:
                t = t + [""] * (n - len(t))
            if len(p) < n:
                p = p + [""] * (n - len(p))
            for i, asig in enumerate(cabecera_cols):
                fila = {
                    'asignatura': _limpia_rotulo(norm_text(asig)),
                    'tareas': t[i] or "",
                    'plan':   p[i] or ""
                }
                # Evita filas completamente vacías
                if fila['asignatura'] or fila['tareas'] or fila['plan']:
                    actual_bloque['filas'].append(fila)
        # Reset estado matriz
        cabecera_cols, tareas_cols, plan_cols = None, None, None

    def _extrae_textos_ths_en_matriz(tr):
        """De la fila .alert.alert-info extrae los <th> (ignorando el primer <td> vacío si lo hay)."""
        # Algunos días empiezan con <td></td> y luego varios <th>
        ths = tr.find_all("th", recursive=False)
        return [th.get_text(" ", strip=True) for th in ths]

    def _extrae_celdas_tds(tr):
        """Devuelve lista de HTML interno de cada <td> en la fila (para alinear por columnas)."""
        celdas = []
        for td in tr.find_all("td", recursive=False):
            celdas.append(_celda_html(td))
        return celdas

    # --- RECORRER TODAS LAS FILAS ---
    for tr in tabla.find_all("tr", recursive=True):
        # A) Cabecera de día
        fecha_dia = _es_fila_header_de_dia(tr)
        if fecha_dia:
            # Antes de cambiar de día, si había una matriz en curso, vuelca sus filas
            _cerrar_bloque_matriz_si_hay()
            # Cierra asignatura vertical pendiente
            if actual_bloque:
                _push_asignatura_vertical()
            actual_bloque = {'fecha': fecha_dia, 'filas': []}
            bloques.append(actual_bloque)
            # limpia estados
            asignatura_actual = None
            cabecera_cols, tareas_cols, plan_cols = None, None, None
            continue

        # Si aún no hay bloque pero empiezan datos, crea uno genérico
        if not actual_bloque:
            actual_bloque = {'fecha': 'Día', 'filas': []}
            bloques.append(actual_bloque)

        # ¿Fila de cabecera MATRIZ (alert alert-info)?
        classes = (tr.get("class") or [])
        if any(c in ("alert", "alert-info") for c in classes):
            # Si ya había una matriz en curso, vuelca antes de arrancar otra
            _cerrar_bloque_matriz_si_hay()
            cabecera_cols = _extrae_textos_ths_en_matriz(tr)
            tareas_cols, plan_cols = None, None
            # También cerramos cualquier asignatura vertical pendiente
            _push_asignatura_vertical()
            continue

        # Detectar th/td de la fila
        ths = tr.find_all("th", recursive=False)
        tds = tr.find_all("td", recursive=False)

        # B) Si estamos en formato MATRIZ (hay cabecera_cols) y la fila tiene un <th> con 'Tareas' o 'Planificación'
        if cabecera_cols and ths:
            rot = _limpia_rotulo(norm_text(ths[0].get_text(" ", strip=True))).lower()
            if rot in {"tareas", "planificación", "planificacion"}:
                celdas = _extrae_celdas_tds(tr)
                if rot == "tareas":
                    tareas_cols = celdas
                else:
                    plan_cols = celdas
                # No volcamos todavía hasta que venga el siguiente día o nueva cabecera de matriz.
                continue
            # Si llega otra cosa con TH y tenemos matriz activa, puede ser un nuevo bloque; volcamos lo que hubiera.
            if cabecera_cols:
                _cerrar_bloque_matriz_si_hay()

        # C) Formato VERTICAL (asignatura por fila)
        if ths:
            rotulo_raw = norm_text(ths[0].get_text(" ", strip=True))
            rotulo = _limpia_rotulo(rotulo_raw).lower()

            if rotulo in {"tareas", "planificación", "planificacion"}:
                if not asignatura_actual:
                    asignatura_actual = {'asignatura': 'General', 'tareas': '', 'plan': ''}
                contenido = _celda_html(tds[0]) if len(tds) >= 1 else ""
                if rotulo == "tareas":
                    asignatura_actual['tareas'] = (asignatura_actual.get('tareas') + "<br>" if asignatura_actual.get('tareas') else "") + contenido
                else:
                    asignatura_actual['plan'] = (asignatura_actual.get('plan') + "<br>" if asignatura_actual.get('plan') else "") + contenido
                continue

            # Nueva asignatura vertical
            _push_asignatura_vertical()
            if rotulo in {"notas personales", "notas", "observaciones"}:
                asignatura_actual = None
                continue

            asignatura_actual = {'asignatura': _limpia_rotulo(rotulo_raw), 'tareas': '', 'plan': ''}
            if tds:
                if len(tds) == 1:
                    asignatura_actual['tareas'] = _celda_html(tds[0])
                else:
                    asignatura_actual['tareas'] = _celda_html(tds[0])
                    asignatura_actual['plan']   = _celda_html(tds[1])
            continue

        # Fila sin TH (rara) -> apéndice a 'tareas' vertical
        if tds and asignatura_actual:
            cont = _celda_html(tds[0])
            asignatura_actual['tareas'] = (asignatura_actual.get('tareas') + "<br>" if asignatura_actual.get('tareas') else "") + cont

    # Final del recorrido: cerrar pendientes
    _cerrar_bloque_matriz_si_hay()
    if actual_bloque:
        _push_asignatura_vertical()

    # Filtra bloques sin filas
    return [b for b in bloques if b['filas']]

def _thead_html():
    return (
        "<thead>"
        "<tr>"
        "<th>Asignatura</th>"
        "<th>Tareas</th>"
        "<th>Planificación</th>"
        "</tr>"
        "</thead>"
    )

def render_tabla_por_dia(fecha: str, filas: list[dict]) -> str:
    filas_html = []
    for f in filas:
        filas_html.append(
            "<tr>"
            f"<th>{f['asignatura']}</th>"
            f"<td>{f.get('tareas','')}</td>"
            f"<td>{f.get('plan','')}</td>"
            "</tr>"
        )
    tabla = (
        "<table style='width:100%; border-collapse:collapse;' class='table table-hover table-bordered'>"
        + _thead_html() +
        "<tbody>" + "".join(filas_html) + "</tbody>"
        "</table>"
    )
    # 🔽 antes ponías "Día {fecha}"
    titulo = _titulo_dia_con_nombre(fecha)
    return f"<h2 style='font-weight:bold; margin:16px 0 8px;'>{titulo}</h2>{tabla}"

def render_bloques(bloques: list[dict]) -> str:
    return "".join(render_tabla_por_dia(b['fecha'], b['filas']) for b in bloques)

# ========= FIN NUEVO =========


try:
    # --- Login ---
    driver.get('https://lasallesanrafael.sallenet.org')
    time.sleep(3)
    username_field = driver.find_element(By.ID, 'username')
    password_field = driver.find_element(By.ID, 'password')

    # *** Manteniendo como lo tenías ***
    username_field.send_keys('davidpaniaguadominguez')
    password_field.send_keys('Valhalla99!')

    driver.find_element(By.ID, 'loginbtn').click()
    time.sleep(10)
    print("Login exitoso")

    # --- Padres -> Trabajo ---
    driver.get('https://lasallesanrafael.sallenet.org/mod/sallenet/modulos/padres/')
    time.sleep(10)
    print("Navegación a 'padres' OK")

    enlace_trabajo = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Trabajo')]"))
    )
    driver.execute_script("arguments[0].click();", enlace_trabajo)
    esperar_tabla()

    # --- Fecha hoy (Europe/Madrid) ---
    try:
        from zoneinfo import ZoneInfo
        hoy_str = dtmod.now(ZoneInfo("Europe/Madrid")).strftime("%d/%m/%Y")
    except Exception:
        hoy_str = dtmod.utcnow().strftime("%d/%m/%Y")
    print(f"[DBG] Hoy: {hoy_str}")

    # 1) Capturar SEMANA primero
    print("[DBG] Cargando VISTA SEMANAL…")
    tabla_semana_html = click_vista_semanal() or get_tabla_html()  # click y leer
    tabla_semana_html = get_tabla_html()  # asegurar lectura tras carga
    print("[DBG] Semanal capturada ✅")

    # 2) Cambiar a DIARIA y capturar
    print("[DBG] Cargando VISTA DIARIA…")
    tabla_dia_html = None
    click_vista_diaria()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'fecha_incidencia')))
    set_fecha_en_input_y_disparar_change(hoy_str)
    esperar_header_de_fecha(hoy_str, timeout=25)
    tabla_dia_html = get_tabla_html()
    print("[DBG] Diaria capturada ✅")

    # --- Email ---
    estilos_css = """
      <style>
        /* Paleta */
        :root {
          --azul-suave: #eaf3ff;   /* azul clarito */
          --borde: #dde6f0;
        }

        table { width: 100%; border-collapse: collapse; }
        th, td {
          border: 1px solid var(--borde);
          text-align: left;
          padding: 10px;
          vertical-align: top;
          background: #fff;              /* base: blanco */
        }

        /* Cabecera (primera fila) en azul claro */
        thead th {
          background: var(--azul-suave);
          font-weight: 700;
        }

        /* Primera columna (las <th> del cuerpo) en azul claro */
        tbody th {
          background: var(--azul-suave);
          width: 26%;
          font-weight: 600;
        }

        /* Quitamos zebra striping si lo hubiera */
        tr:nth-child(even) td, tr:nth-child(even) th {
          background: inherit;
        }

        /* Hover sutil para filas del cuerpo */
        tbody tr:hover td { background: #f7fbff; }
        h1 { margin: 0 0 10px; }
        h2 { margin: 16px 0 8px; }
      </style>
      """

    bloque_meteo = meteo_moncloa_semana_html()

    # --- Limpieza previa (quitar 'Notas personales') ---
    tabla_semana_html = quitar_notas_personales(tabla_semana_html)
    tabla_dia_html    = quitar_notas_personales(tabla_dia_html)

    # ===== AQUI transformamos a tus tablas “como la del tiempo” =====
    bloques_dia    = parsear_bloques_por_dia(tabla_dia_html)
    bloques_semana = parsear_bloques_por_dia(tabla_semana_html)

    tabla_html_modificada  = render_bloques(bloques_dia)    if bloques_dia    else "<p><em>Sin datos para el día.</em></p>"
    tabla_html_modificada2 = render_bloques(bloques_semana) if bloques_semana else "<p><em>Sin datos para la semana.</em></p>"
    # ===== FIN TRANSFORMACIÓN =====

    contenido_html = f"""
    <html>
    <head>{estilos_css}</head>
    <body>
        <h1 style="font-weight:bold;">DIA ACTUAL</h1>
        {tabla_html_modificada}
        <h1 style="font-weight:bold;">SEMANA COMPLETA</h1>
        {tabla_html_modificada2}
        {bloque_meteo}
    </body>
    </html>
    """

    contenido_html_inline = css_inline.inline(contenido_html)
    contenido_html_min = minify_html(contenido_html_inline)

    dia_actual_str = dtmod.now().strftime("%d-%m-%Y")
    subject = "Calendario de tareas La Salle 1B " + dia_actual_str

    yag = yagmail.SMTP('elpaniaguado@gmail.com')  # o como lo tuvieras
    destinatarios3 = [
        'elpaniaguado@gmail.com',
        'cquinteromoreno@gmail.com',
        'sharonrdc@yahoo.es',
        'abdjimenez@hotmail.com',
        'paradinasmarcos@gmail.com',
        'paulaherediaalcantara@gmail.com',
        'mariofvilladangos@hotmail.com',
        'adrian.alvarez.munoz@gmail.com',
        'nu.abril@gmail.com',
        'lorenitavelayos@gmail.com',
        'judiith22@gmail.com',
        'aransay13@gmail.com',
        'emmaedani@gmail.com',
        'anaisabelmas@hotmail.com',
        'juandiazlorenzo@hotmail.com',
        "lorena.torresc85@gmail.com"
    ]
    destinatarios = ['elpaniaguado@gmail.com']  # para pruebas

    try:
        yag.send(
            to=destinatarios,
            subject=subject,
            contents=[yagmail.raw(contenido_html_min)]
        )
        print("Correo enviado con éxito a todos los destinatarios.")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

except Exception as e:
    print("Fallo:", e)
    dump_debug("fatal")
    try:
        for entry in driver.get_log('browser'):
            print(f"[BROWSER] {entry.get('level')}: {entry.get('message')}")
    except Exception:
        pass
    raise
finally:
    driver.quit()
    print("Fin.")





Empiezo


ModuleNotFoundError: No module named 'yagmail'